In [42]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

In [8]:
df_train = pd.read_csv("../data/train_new.csv", sep="|")
df_train_val = pd.read_csv("../data/val_new.csv", sep="|")
df_train_complete = pd.read_csv("../data/train.csv", sep="|")
df_test = pd.read_csv("../data/test.csv", sep="|")
df_train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,4,828,66.56,7,4,3,0.007246,0.080386,1.166667,0
1,1,1612,31.34,2,4,3,0.008685,0.019442,0.142857,0
2,3,848,52.37,2,4,0,0.022406,0.061757,0.105263,0
3,1,321,76.03,8,7,2,0.071651,0.236854,0.347826,0
4,1,660,6.06,3,7,1,0.027273,0.009182,0.166667,0


In [9]:
print("Train_new size:", df_train.size)
print("Train_val size:", df_train_val.size)
print("Train_complete size:", df_train_complete.size)
print("Test size:", df_test.size)

Train_new size: 15030
Train_val size: 3760
Train_complete size: 18790
Test size: 4483089


### The following cell defines our loss function, based on the table shown in the accompagning pdf

In [47]:
def my_custom_score_function(y, y_pred):
    loss = 0.0
    for index, item in enumerate(y):
        if y[index] == 0: 
            if y_pred[index] == 0:
                loss += 0.0
            else:
                loss -= 25.0
        else:
            if y_pred[index] == 0:
                loss -= 5.0
            else:
                loss += 5.0
    return loss

In [60]:
def custom_score_function(y, y_pred):
    return metrics.fbeta_score(y, y_pred, beta=2)

### Try some training below

In [61]:
df_train_labels = df_train["fraud"]
df_train_values = df_train.drop(columns=["fraud"])
df_train_val_labels = df_train_val["fraud"]
df_train_val_values = df_train_val.drop(columns=["fraud"])

In [62]:
scorer = metrics.make_scorer(my_custom_loss_function)

#### AdaBoostClassifier

In [69]:
import sklearn.ensemble as ensemble
abc = ensemble.AdaBoostClassifier() 
abc.fit(df_train_values, df_train_labels)
y_pred = abc.predict(df_train_val_values)
score_dmc = my_custom_score_function(df_train_val_labels, y_pred)
score_fbeta2 = custom_score_function(df_train_val_labels, y_pred)
print("DMC Score: ", score_dmc)
print("FBeta2 Score: ", score_fbeta2)

DMC Score:  -140.0
FBeta2 Score:  0.6578947368421053


#### Naive Bayes

In [71]:
import sklearn.naive_bayes as nb
gnb = nb.GaussianNB()
gnb.fit(df_train_values, df_train_labels)
y_pred_gnb = gnb.predict(df_train_val_values)
score_dmc_gnb = my_custom_score_function(df_train_val_labels, y_pred_gnb)
score_fbeta2 = custom_score_function(df_train_val_labels, y_pred_gnb)
print("DMC Score: ", score_dmc)
print("FBeta2 Score: ", score_fbeta2)

DMC Score:  -140.0
FBeta2 Score:  0.5833333333333333
